In [2]:
import numpy as np
import itertools
import random
from copy import deepcopy

In [ ]:
class ant:
    def __init__(self, n, m):
        self.size = n * m
        self.path = [randint(0, self.size-1)]
        self.n = n
        self.m = m
        
    def nextMoves(self, a):
        new = []
        return new.copy()

    def distMove(self, a):
        dummy=ant(self.n, self.m)
        dummy.path=self.path.copy()
        dummy.path.append(a)
        return (9-len(dummy.nextMoves(a)))
        
    def addMove(self, q0, trace, alpha, beta):
        # adauga o noua pozitie in solutia furnicii daca este posibil
        p = [0 for i in range(self.size)]
        # pozitiile ce nu sunt valide vor fi marcate cu zero
        nextSteps=self.nextMoves(self.path[len(self.path)-1]).copy()
        # determina urmatoarele pozitii valide in nextSteps
        # daca nu avem astfel de pozitii iesim 
        if (len(nextSteps) == 0):
            return False
        # punem pe pozitiile valide valoarea distantei empirice
        for i in nextSteps:
            p[i] = self.distMove(i)
        # calculam produsul trace^alpha si vizibilitate^beta
        p=[ (p[i]**beta)*(trace[self.path[-1]][i]**alpha) for i in range(len(p))]
        if (random()<q0):
            # adaugam cea mai buna dintre mutarile posibile
            p = [ [i, p[i]] for i in range(len(p)) ]
            p = max(p, key=lambda a: a[1])
            self.path.append(p[0])
        else:
            # adaugam cu o probabilitate un drum posibil (ruleta)
            s = sum(p)
            if (s==0):
                return choice(nextSteps)
            p = [ p[i]/s for i in range(len(p)) ]
            p = [ sum(p[0:i+1]) for i in range(len(p)) ]
            r=random()
            i=0
            while (r > p[i]):
                i=i+1
            self.path.append(i)
        return True
                   
    def fitness(self):
        # un drum e cu atat mai bun cu cat este mai lung
        # problema de minimizare, drumul maxim e n * m
        return (self.size-len(self.path)+2)

def epoca(noAnts, n, m, trace, alpha, beta, q0, rho):
    antSet=[ant(n, m) for i in range(noAnts)]
    for i in range(n * m):
        # numarul maxim de iteratii intr-o epoca este lungimea solutiei
        for x in antSet:
            x.addMove(q0, trace, alpha, beta)
    # actualizam trace-ul cu feromonii lasati de toate furnicile
    dTrace=[ 1.0 / antSet[i].fitness() for i in range(len(antSet))]
    for i in range(n * m):
        for j in range (n * m):
            trace[i][j] = (1 - rho) * trace[i][j]
    for i in range(len(antSet)):
        for j in range(len(antSet[i].path)-1):
            x = antSet[i].path[j]
            y = antSet[i].path[j+1]
            trace[x][y] = trace [x][y] + dTrace[i]
    # return best ant path
    f=[ [antSet[i].fitness(), i] for i in range(len(antSet))]
    f=max(f)
    return antSet[f[1]].path
    
def main(n = 8,m = 8,noEpoch = 100,noAnts = 3,alpha = 1.9,beta = 0.9,rho = 0.05,q0 = 0.5):
    sol=[]
    bestSol=[]
    trace=[[1 for i in range(n * m)] for j in range (n * m)]
    print("Programul ruleaza! Dureaza ceva timp pana va termina!")
    for i in range(noEpoch):
        sol=epoca(noAnts, n, m, trace, alpha, beta, q0, rho).copy()
        if len(sol)>len(bestSol):
            bestSol=sol.copy()
    print ("lungimea celei mai bune solutii depistate la aceasta rulare:", len(bestSol))
    print ("Drumul detectat este:", bestSol)

In [19]:
class Ant:
    def __init__(self, n):
        self.solution = []
        self.n = n
    
    def __intersects(self, row, solution):
        for i in range(len(solution)):
            if len(set(row) & set(solution[i])) != 0:
                return True
        return False
    
    def __uniqueColumn(self, row, solution):
        for r in range(len(solution)):
            for c in range(len(row)):
                if row[c][0] == solution[r][c][0] or row[c][1] == solution[r][c][1]:
                    return False
        return True
    
    def nextMoves(self):
        moves = []
        permutations = list(itertools.permutations([i for i in range(1, self.n + 1)]))
        
        for i in range(len(permutations)):
            for j in range(len(permutations)):
                row = list(zip(permutations[i], permutations[j]))
#                 if not self.__intersects(row, self.solution) and self.__uniqueColumn(row, self.solution):
#                     moves.append(row)
                    
                if not self.__intersects(row, self.solution):
                    moves.append(row)
                
        return moves
    
    def update(self, q0, trace, alpha, beta):
        nextSteps = self.nextMoves(self.solution)
        
        p = [0 for i in range(len(nextSteps))]
 
        if (len(nextSteps) == 0):
            return False
        
        for i in len(nextSteps):
            p[i] = self.distMove(nextSteps[i])
            
        p = [(trace[self.solution][i] ** alpha) * (p[i] ** beta) for i in range(len(p))]
        
        if (random() < q0):
            p = [[i, p[i]] for i in range(len(p))]
            p = max(p, key=lambda a: a[1])
            self.solution.append(p[0])
        else:
            s = sum(p)
            if (s == 0):
                return random.choice(nextSteps)
            
            p = [p[i]/s for i in range(len(p))]
            p = [sum(p[0:i+1]) for i in range(len(p))]
            r = random()
            i = 0
            while (r > p[i]):
                i=i+1
            self.path.append(i)
        return True
    
    
    def distMove(self, nextStep):
        futureAnt = Ant(self.n)
        futureAnt.solution = deepcopy(self.solution)
        futureAnt.solution.append(nextStep)
        return futureAnt.evaluate()
    
    def evaluate(self):
        fitness = 0
        for r in range(len(self.solution)):
            for c in range(self.n):
                if row[c][0] == solution[r][c][0] or row[c][1] == solution[r][c][1]:
                    fitness += 1
        return 1/fitness
    
    def __str__(self):
        if len(self.solution) == 0:
            return "Empty solution"
        
        s = ""
        for i in range(len(self.solution)):
            for j in range(self.n):
                s += str(self.solution[i][j]) +  " "
            s += "\n"
            
        return s

In [2]:
class Controller:
    def __init__(self, population):
        self.population = population
        self.trace = []
        
        self.q0 = 0
        self.alpha = 0
        self.beta = 0
    
def iteration(noAnts, n, trace, alpha, beta, q0, rho):
    antSet = [Ant(n) for i in range(noAnts)]
    for i in range(n):
        for x in antSet:
            x.addMove(q0, trace, alpha, beta)

    dTrace = [1.0 / antSet[i].evaluate() for i in range(len(antSet))]
    for i in range(n):
        for j in range (n):
            trace[i][j] = (1 - rho) * trace[i][j]
            
    for i in range(len(antSet)):
        for j in range(len(antSet[i].path)-1):
            x = antSet[i].path[j]
            y = antSet[i].path[j+1]
            trace[x][y] = trace [x][y] + dTrace[i]
    # return best ant path
    f=[ [antSet[i].fitness(), i] for i in range(len(antSet))]
    f=max(f)
    return antSet[f[1]].path
    
    def runAlg(self):
        pass
    
    def loadParameters(self, q0, alpha, beta):
        self.q0 = q0
        self.alpha = alpha
        self.beta = beta

In [ ]:
class Problem:
    def __init__(self):
        pass
    
    def loadProblem(self):
        pass